<a href="https://colab.research.google.com/github/AbhayZ1/GNN_Squats/blob/main/GNN_squats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycocotools tensorflow_gnn tf2onnx opencv-python-headless mediapipe

INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.8/836.8 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.4/801.4 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16

In [3]:
!pip uninstall -y tensorflow keras

# # Install specific versions of TensorFlow and Keras
!pip install tensorflow==2.15.0 keras==2.15.0

# Install other required packages
!pip install pycocotools tensorflow_gnn tf2onnx opencv-python-headless mediapipe

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled 

In [2]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
import numpy as np
import tf2onnx
from pycocotools.coco import COCO
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow

# Initialize COCO API
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O annotations.zip
!unzip -q annotations.zip
annFile = 'annotations/person_keypoints_train2017.json'
coco = COCO(annFile)

# Get all image ids with person annotations
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds)

def get_keypoints(img_id):
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(ann_ids)
    keypoints = [ann['keypoints'] for ann in anns if ann['num_keypoints'] > 0]
    return keypoints

def process_keypoints(keypoints):
    return np.array(keypoints).reshape(-1, 3)[:, :2]

def create_graph_tensor(keypoints):
    nodes = process_keypoints(keypoints)
    node_set = tfgnn.NodeSet.from_fields(
        sizes=[len(nodes)],
        features={'features': nodes.astype(np.float32)}
    )
    return tfgnn.GraphTensor.from_pieces(
        node_sets={"nodes": node_set},
        edge_sets={"edges": tfgnn.EdgeSet.from_fields(
            sizes=[len(nodes)],
            adjacency=tfgnn.Adjacency.from_indices(
                source=("nodes", np.arange(len(nodes))),
                target=("nodes", np.arange(len(nodes)))
            )
        )}
    )


# Create graph tensors
graph_tensors = []
for img_id in imgIds[:10000]:  # Limit to 1000 images for this example
    img_keypoints = get_keypoints(img_id)
    if img_keypoints:
        graph_tensors.extend([create_graph_tensor(kp) for kp in img_keypoints])

print(f"Created {len(graph_tensors)} graph tensors")

# Create dummy labels (0 or 1) for each graph tensor
labels = np.random.randint(0, 2, size=(len(graph_tensors),))


--2025-02-26 18:56:41--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.202.177, 3.5.25.48, 54.231.167.81, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.202.177|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations.zip’

annotations.zip     100%[===================>] 241.19M  49.6MB/s    in 5.2s    

2025-02-26 18:56:46 (46.8 MB/s) - ‘annotations.zip’ saved [252907541/252907541]

replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace annotations/instances_val2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace annotations/captions_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace annotations/captions_val2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace annotations/person_keypoints_train2017.json? [y]es, [n]o, [A]ll, [N]one, [

In [3]:
len(imgIds)

64115

In [8]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
import numpy as np
import cv2
import mediapipe as mp
from pycocotools.coco import COCO
import tf2onnx

# MediaPipe setup
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# COCO dataset setup
annFile = 'annotations/person_keypoints_train2017.json'
coco = COCO(annFile)
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds)

def create_graph_tensor(keypoints):
    node_features = np.array(keypoints).reshape(-1, 3)
    return tfgnn.GraphTensor.from_pieces(
        context=tfgnn.Context.from_fields(features={}),
        node_sets={'nodes': tfgnn.NodeSet.from_fields(
            sizes=[len(node_features)],
            features={'features': node_features.astype(np.float32)})},
        edge_sets={'edges': tfgnn.EdgeSet.from_fields(
            sizes=[len(node_features)],
            adjacency=tfgnn.Adjacency.from_indices(
                source=('nodes', np.arange(len(node_features))),
                target=('nodes', np.arange(len(node_features)))))}
    )

# Define label_squat function
def label_squat(keypoints):
    # COCO keypoint order: [nose, left_eye, right_eye, left_ear, right_ear, left_shoulder, right_shoulder,
    #                       left_elbow, right_elbow, left_wrist, right_wrist, left_hip, right_hip,
    #                       left_knee, right_knee, left_ankle, right_ankle]
    hip_y = (keypoints[11] + keypoints[12]) / 2  # Average of left and right hip y-coordinates
    knee_y = (keypoints[13] + keypoints[14]) / 2  # Average of left and right knee y-coordinates
    ankle_y = (keypoints[15] + keypoints[16]) / 2  # Average of left and right ankle y-coordinates

    if hip_y > knee_y and knee_y > ankle_y:
        return 1  # Correct squat
    else:
        return 0  # Incorrect squat

# Data preparation
X = []
y = []

for img_id in imgIds:  # Limit to 1000 images for this example
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(ann_ids)
    for ann in anns:
        if ann['num_keypoints'] > 0:
            keypoints = ann['keypoints']
            # Extract features directly instead of creating a GraphTensor
            features = np.array(keypoints).reshape(-1, 3).astype(np.float32)
            X.append(features)
            y.append(label_squat(keypoints))

X = np.array(X)
y = np.array(y)

print(f"Number of samples: {len(X)}")
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# The rest of the code (model definition, training, ONNX conversion, etc.) remains the same

# MediaPipe setup
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def extract_keypoints(results):
    keypoints = []
    for landmark in mp_pose.PoseLandmark:
        point = results.pose_landmarks.landmark[landmark]
        keypoints.extend([point.x, point.y, point.z])
    return np.array(keypoints)

class ImprovedGNN(tf.keras.Model):
    def __init__(self, hidden_dim):
        super(ImprovedGNN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(hidden_dim, activation="relu")
        self.dense2 = tf.keras.layers.Dense(hidden_dim, activation="relu")
        self.global_pool = tf.keras.layers.GlobalAveragePooling1D()
        self.classifier = tf.keras.layers.Dense(1, activation="sigmoid")

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.global_pool(x)
        return self.classifier(x)

# Initialize and compile model
model = ImprovedGNN(hidden_dim=64)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


# ONNX conversion
spec = (tf.TensorSpec((None, 33, 3), tf.float32, name="input"),)
output_path = "squat_classifier.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)

# Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")

# Video processing function
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(cap.get(3)), int(cap.get(4))))

    with mp_pose.Pose() as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if results.pose_landmarks:
                keypoints = extract_keypoints(results)
                graph_tensor = create_graph_tensor(keypoints)
                prediction = model.predict(np.expand_dims(graph_tensor, axis=0))

                squat_class = "Correct" if prediction[0][0] > 0.5 else "Incorrect"
                cv2.putText(frame, f"Squat: {squat_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            out.write(frame)

    cap.release()
    out.release()

# Example usage
# process_video('input_squat_video.mp4', 'output_squat_video.mp4')


loading annotations into memory...
Done (t=12.65s)
creating index...
index created!
Number of samples: 149813
Shape of X: (149813, 17, 3)
Shape of y: (149813,)
Epoch 1/10
3746/3746 [==============================] - 13s 3ms/step - loss: 0.0988 - accuracy: 0.9824 - val_loss: 0.0826 - val_accuracy: 0.9829
Epoch 2/10
3746/3746 [==============================] - 13s 4ms/step - loss: 0.0847 - accuracy: 0.9826 - val_loss: 0.0754 - val_accuracy: 0.9829
Epoch 3/10
3746/3746 [==============================] - 15s 4ms/step - loss: 0.0810 - accuracy: 0.9826 - val_loss: 0.0771 - val_accuracy: 0.9829
Epoch 4/10
3746/3746 [==============================] - 11s 3ms/step - loss: 0.0785 - accuracy: 0.9826 - val_loss: 0.0752 - val_accuracy: 0.9829
Epoch 5/10
3746/3746 [==============================] - 14s 4ms/step - loss: 0.0777 - accuracy: 0.9826 - val_loss: 0.0737 - val_accuracy: 0.9829
Epoch 6/10
3746/3746 [==============================] - 14s 4ms/step - loss: 0.0769 - accuracy: 0.9826 - val_loss: 

In [9]:
# Prepare data from COCO dataset
X = []
y = []
for graph_tensor in graph_tensors:
    # Extract node features from the graph tensor
    node_features = graph_tensor.node_sets['nodes']['features'].numpy()
    # Flatten the node features
    flattened_features = node_features.flatten()
    X.append(flattened_features)

X = np.array(X)
y = labels

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

if len(X) == 0 or len(y) == 0:
    raise ValueError("X or y is empty. Check data preparation steps.")

# Ensure all samples have the same number of features
feature_length = X.shape[1]
X = np.array([x for x in X if len(x) == feature_length])
y = y[:len(X)]

print(f"Updated shape of X: {X.shape}")
print(f"Updated shape of y: {y.shape}")

# Convert to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(buffer_size=len(X))

# Split dataset
train_size = int(0.8 * len(X))
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# Batch the datasets
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# Check if datasets are empty
if tf.data.experimental.cardinality(train_dataset).numpy() == 0:
    raise ValueError("Training dataset is empty")
if tf.data.experimental.cardinality(test_dataset).numpy() == 0:
    raise ValueError("Test dataset is empty")

print(f"Number of batches in train_dataset: {tf.data.experimental.cardinality(train_dataset).numpy()}")
print(f"Number of batches in test_dataset: {tf.data.experimental.cardinality(test_dataset).numpy()}")


class ImprovedGNN(tf.keras.Model):
    def __init__(self, hidden_dim):
        super(ImprovedGNN, self).__init__()
        self.node_update = tf.keras.layers.Dense(hidden_dim, activation='relu')
        self.global_pooling = tf.keras.layers.GlobalAveragePooling1D()
        self.classifier = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # Reshape inputs to (batch_size, num_nodes, features_per_node)
        x = tf.reshape(inputs, (-1, 17, 2))  # Adjust based on COCO keypoints

        # Node update
        node_features = self.node_update(x)

        # Global pooling
        global_features = self.global_pooling(node_features)

        # Classification
        return self.classifier(global_features)

# Initialize and compile model
model = ImprovedGNN(hidden_dim=32)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Print model summary and final accuracy
model.summary()
print(f"Final training accuracy: {history.history['accuracy'][-1]:.2f}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.2f}")

def analyze_squat(keypoints):
    hip_y = (keypoints[11, 1] + keypoints[12, 1]) / 2  # Adjust indices based on COCO keypoints
    knee_y = (keypoints[13, 1] + keypoints[14, 1]) / 2
    ankle_y = (keypoints[15, 1] + keypoints[16, 1]) / 2

    if hip_y > knee_y:
        return "Incorrect: Hips not low enough"
    elif knee_y < ankle_y:
        return "Incorrect: Knees too far forward"
    else:
        return "Correct form"

def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            if results.pose_landmarks:
                keypoints = np.array([[lm.x, lm.y] for lm in results.pose_landmarks.landmark])

                input_data = keypoints.flatten().reshape(1, -1)  # Flatten for input
                prediction = model.predict(input_data)
                squat_class = "Correct" if prediction[0][0] > 0.5 else "Incorrect"

                analysis = analyze_squat(keypoints)

                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                cv2.putText(frame, f"Squat: {squat_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, analysis, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            out.write(frame)

    cap.release()
    out.release()

# Example usage
# process_video('path_to_input_video.mp4', 'path_to_output_video.mp4')


Shape of X: (23193, 34)
Shape of y: (23193,)
Updated shape of X: (23193, 34)
Updated shape of y: (23193,)
Number of batches in train_dataset: 580
Number of batches in test_dataset: 145
Epoch 1/10
580/580 [==============================] - 2s 2ms/step - loss: 2.8042 - accuracy: 0.5004 - val_loss: 0.7149 - val_accuracy: 0.4990
Epoch 2/10
580/580 [==============================] - 1s 2ms/step - loss: 0.7303 - accuracy: 0.5018 - val_loss: 0.7057 - val_accuracy: 0.5040
Epoch 3/10
580/580 [==============================] - 1s 2ms/step - loss: 0.7257 - accuracy: 0.5037 - val_loss: 0.7252 - val_accuracy: 0.5025
Epoch 4/10
580/580 [==============================] - 1s 2ms/step - loss: 0.7320 - accuracy: 0.5005 - val_loss: 0.6966 - val_accuracy: 0.5096
Epoch 5/10
580/580 [==============================] - ETA: 0s - loss: 0.7326 - accuracy: 0.5019

KeyboardInterrupt: 